In [42]:
# %% [markdown]
# Изолинии плотности по точкам, вписанные в контуры квартала (без утечек за границу)
# Пути берем из вашего скрипта:
#   zones  -> /root/globalmapper_learning/zones.geojson
#   points -> /root/globalmapper_learning/out_2/blocks_infer_centroids.geojson
# Дефолтная СК: EPSG:32636 (метры).
#
# Как отключить влияние 'e':
#   1) Поставьте USE_E_WEIGHTS = False  (веса = 1 для всех точек), ИЛИ
#   2) Укажите WEIGHT_COLUMN = None.
#
# Требуется: geopandas, shapely, numpy, pandas, scikit-image, scipy

# !pip install geopandas shapely pyproj numpy pandas scikit-image scipy --quiet

import math
from pathlib import Path
from typing import List, Optional, Tuple

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
from shapely.prepared import prep
from scipy.ndimage import gaussian_filter
from skimage.measure import find_contours

# ---------------------
# Параметры
# ---------------------
PATH_ZONES  = Path("/root/globalmapper_learning/zones.geojson")
PATH_POINTS = Path("/root/globalmapper_learning/out_2/infer_centroids.geojson")
OUT_PATH    = Path("/root/globalmapper_learning/out_2/isolines.geojson")

CRS_EPSG        = 32636
GRID_SIZE_M     = 15.0          # шаг растра, м
BANDWIDTH_M     = 10.0          # σ Гаусса, м
LEVEL_QUANTILES = [0.60, 0.75, 0.85, 0.92, 0.97]
MIN_LINE_LEN_M  = 2 * GRID_SIZE_M
MAX_LEVELS      = 12

# Управление весами 'e'
USE_E_WEIGHTS   = True          # ← поставьте False, чтобы полностью убрать влияние e
WEIGHT_COLUMN   = None          # если хотите явно указать колонку веса, например "e"; None = авто-поиск
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

# ---------------------
# Утилиты
# ---------------------
def ensure_epsg_32636(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """Работаем в EPSG:32636: если crs отсутствует — назначаем, если иная — перепроецируем."""
    if gdf.crs is None:
        return gdf.set_crs(epsg=CRS_EPSG, allow_override=True)
    if gdf.crs.to_epsg() != CRS_EPSG:
        return gdf.to_crs(epsg=CRS_EPSG)
    return gdf

def extract_weights(gdf_pts: gpd.GeoDataFrame, use_e: bool = True, weight_col: Optional[str] = None) -> np.ndarray:
    """Вернуть веса точек. Если use_e=False → все единицы. Иначе — берем колонку 'e' (или заданную), нормируем в [1,2]."""
    if not use_e:
        return np.ones(len(gdf_pts), dtype=float)
    col = weight_col
    if col is None:
        for c in ["e","e_i","E","e_pred","existence","existence_score"]:
            if c in gdf_pts.columns:
                col = c
                break
    if col is None:
        return np.ones(len(gdf_pts), dtype=float)
    w = pd.to_numeric(gdf_pts[col], errors="coerce").fillna(0.0).to_numpy(dtype=float)
    if w.size:
        w_min, w_max = np.nanmin(w), np.nanmax(w)
        if w_max > w_min:
            w = (w - w_min) / (w_max - w_min)
        w = np.clip(w, 0.0, 1.0)
        w = 1.0 + w  # итог 1..2
    else:
        w = np.ones(len(gdf_pts), dtype=float)
    return w

def cell_centers_mask(poly, xedges, yedges):
    xs = (xedges[:-1] + xedges[1:]) * 0.5
    ys = (yedges[:-1] + yedges[1:]) * 0.5
    XX, YY = np.meshgrid(xs, ys)  # H x W
    centers = np.column_stack([XX.ravel(), YY.ravel()])
    m = np.fromiter((poly.covers(Point(xy)) for xy in centers), dtype=bool).reshape(len(ys), len(xs))
    return m

def density_grid_for_block(poly, px, py, pw, grid_m: float, bandwidth_m: float):
    """
    Растр внутри квартала с корректным поведением у границ:
      1) В гистограмму попадают только точки, которые действительно лежат в квартале.
      2) Сглаживание Гауссом нормируется на «маску внутри» (edge correction), чтобы не занижать у границ.
      Возврат: (density HxW, xedges, yedges, inside_mask)
    """
    minx, miny, maxx, maxy = poly.bounds
    W = max(3, int(math.ceil((maxx - minx) / grid_m)))
    H = max(3, int(math.ceil((maxy - miny) / grid_m)))

    # Регулярная сетка по bbox (геометрия нужна для покрытия именно полигона)
    xedges = np.linspace(minx, maxx, W+1)
    yedges = np.linspace(miny, maxy, H+1)

    # Маска ячеек строго внутри полигона
    inside_mask = cell_centers_mask(poly, xedges, yedges)

    # Точки: сначала bbox-фильтр, затем точная проверка внутри полигона
    pr = prep(poly)
    in_bbox = (px >= minx) & (px <= maxx) & (py >= miny) & (py <= maxy)
    if np.any(in_bbox):
        sel = np.fromiter((pr.covers(Point(x, y)) for x, y in zip(px[in_bbox], py[in_bbox])), dtype=bool)
        px_in = px[in_bbox][sel]
        py_in = py[in_bbox][sel]
        pw_in = pw[in_bbox][sel]
    else:
        px_in = py_in = pw_in = np.array([], dtype=float)

    # 2D-гистограмма ТОЛЬКО по точкам внутри квартала
    hist, _, _ = np.histogram2d(
        px_in, py_in,
        bins=[W, H],
        range=[[minx, maxx], [miny, maxy]],
        weights=pw_in
    )
    counts = hist.T  # H x W

    # Гауссово сглаживание + edge correction (делим на сглаженную маску внутри)
    sigma_cells = max(0.5, float(bandwidth_m / grid_m))
    smooth_counts = gaussian_filter(counts, sigma=sigma_cells, mode="nearest")
    smooth_mask   = gaussian_filter(inside_mask.astype(float), sigma=sigma_cells, mode="nearest")
    smooth_mask   = np.maximum(smooth_mask, 1e-9)  # защита от деления на 0
    smooth_corr   = smooth_counts / smooth_mask

    # Плотность (ед./м²); вне полигона ставим чуть ниже минимума внутри
    cell_area = grid_m * grid_m
    density = smooth_corr / max(cell_area, 1.0)
    vals_in = density[inside_mask]
    if vals_in.size:
        min_in = float(vals_in.min())
        density = np.where(inside_mask, density, min_in - 1e-9)
    else:
        density = np.where(inside_mask, density, -np.inf)

    return density, xedges, yedges, inside_mask

def build_levels_from_quantiles(vals: np.ndarray, quantiles: List[float]) -> List[float]:
    vals = np.asarray(vals)
    vals = vals[np.isfinite(vals)]
    if vals.size == 0:
        return []
    vmin, vmax = float(vals.min()), float(vals.max())
    if not np.isfinite(vmin) or not np.isfinite(vmax) or vmin == vmax:
        return []
    lvl_list = []
    for q in quantiles:
        q = float(np.clip(q, 0.0, 1.0))
        lvl = float(np.quantile(vals, q))
        if vmin < lvl < vmax and (not lvl_list or abs(lvl - lvl_list[-1]) > 1e-12):
            lvl_list.append(lvl)
    if len(lvl_list) == 0:
        lvl_list = list(np.linspace(vmin + 1e-9, vmax - 1e-9, 5))
    if len(lvl_list) > MAX_LEVELS:
        step = max(1, len(lvl_list)//MAX_LEVELS)
        lvl_list = lvl_list[::step][:MAX_LEVELS]
    return lvl_list

def contours_for_levels(density, xedges, yedges, levels, min_len_m):
    lines = []
    dx = (xedges[1] - xedges[0])
    dy = (yedges[1] - yedges[0])
    def ij_to_xy(ii, jj):
        x = xedges[0] + jj * dx
        y = yedges[0] + ii * dy
        return x, y
    for lvl in levels:
        try:
            paths = find_contours(density, level=lvl)
        except Exception:
            continue
        for path in paths:
            if len(path) < 2:
                continue
            coords = [ij_to_xy(i, j) for (i, j) in path]
            geom = LineString(coords)
            if geom.is_valid and geom.length >= min_len_m:
                lines.append((geom, lvl))
    return lines

# ---------------------
# Загрузка данных и CRS
# ---------------------
gdf_z = gpd.read_file(PATH_ZONES)
gdf_p = gpd.read_file(PATH_POINTS)

# Точки и веса (уберите влияние e: USE_E_WEIGHTS=False или WEIGHT_COLUMN=None)
px = gdf_p.geometry.x.to_numpy()
py = gdf_p.geometry.y.to_numpy()
pw = extract_weights(gdf_p, use_e=USE_E_WEIGHTS, weight_col=WEIGHT_COLUMN)

print(f"CRS: zones={gdf_z.crs}, points={gdf_p.crs}")
print(f"Counts: zones={len(gdf_z)}, points={len(gdf_p)}")

# ---------------------
# Основной цикл по кварталам
# ---------------------
out_records = []
zone_id_col = next((c for c in ["id","zone_id","ZONE_ID","zone","name"] if c in gdf_z.columns), None)

for idx, row in gdf_z.iterrows():
    poly = row.geometry
    if poly is None or poly.is_empty:
        continue
    zid = row.get(zone_id_col, idx) if zone_id_col else idx

    density, xedges, yedges, inside_mask = density_grid_for_block(poly, px, py, pw, GRID_SIZE_M, BANDWIDTH_M)

    vals_in = density[inside_mask]
    finite_pos = vals_in[np.isfinite(vals_in) & (vals_in > 0)]
    print(f"[zone {zid}] cells_in={inside_mask.sum()}, pos_cells={finite_pos.size}, "
          f"density[min,max]=({np.nanmin(vals_in):.4g}, {np.nanmax(vals_in):.4g})")

    levels = build_levels_from_quantiles(finite_pos, LEVEL_QUANTILES)
    if not levels:
        vmn, vmx = float(np.nanmin(vals_in)), float(np.nanmax(vals_in))
        if np.isfinite(vmn) and np.isfinite(vmx) and vmn < vmx:
            levels = [vmn + 0.5*(vmx - vmn)]
        else:
            print(f"  [skip] нет валидного диапазона плотности для изолиний")
            continue

    lines = contours_for_levels(density, xedges, yedges, levels, MIN_LINE_LEN_M)
    if not lines and finite_pos.size:
        a, b = np.quantile(finite_pos, 0.05), np.quantile(finite_pos, 0.95)
        if a < b:
            tight_levels = list(np.linspace(a + 1e-9, b - 1e-9, min(5, MAX_LEVELS)))
            lines = contours_for_levels(density, xedges, yedges, tight_levels, MIN_LINE_LEN_M)

    for geom, lvl in lines:
        inter = geom.intersection(poly)
        if inter.is_empty:
            continue
        if inter.geom_type == "LineString":
            geoms = [inter]
        elif inter.geom_type == "MultiLineString":
            geoms = list(inter.geoms)
        else:
            continue
        for g in geoms:
            if g.length >= MIN_LINE_LEN_M:
                out_records.append({
                    "zone_id": zid,
                    "level": float(lvl),
                    "grid_m": float(GRID_SIZE_M),
                    "bandwidth_m": float(BANDWIDTH_M),
                    "use_e": bool(USE_E_WEIGHTS),
                    "geometry": g
                })

# ---------------------
# Сохранение
# ---------------------
if out_records:
    gdf_out = gpd.GeoDataFrame(out_records, geometry="geometry", crs=gdf_z.crs)
else:
    gdf_out = gpd.GeoDataFrame(
        {"zone_id": pd.Series(dtype="int64"),
         "level": pd.Series(dtype="float64"),
         "grid_m": pd.Series(dtype="float64"),
         "bandwidth_m": pd.Series(dtype="float64"),
         "use_e": pd.Series(dtype="bool"),
         "geometry": gpd.GeoSeries(dtype="geometry")},
        geometry="geometry",
        crs=gdf_z.crs
    )

gdf_out_wgs = gdf_out.to_crs(4326)
gdf_out_wgs.to_file(OUT_PATH, driver="GeoJSON")
print(f"Готово: {OUT_PATH} | изолиний: {len(gdf_out_wgs)}")
print(f"GRID={GRID_SIZE_M} м, BANDWIDTH={BANDWIDTH_M} м, уровни={LEVEL_QUANTILES}, USE_E_WEIGHTS={USE_E_WEIGHTS}")


CRS: zones=EPSG:32636, points=EPSG:32636
Counts: zones=3, points=6678
[zone 63] cells_in=3383, pos_cells=3290, density[min,max]=(0, 0.1549)
[zone 65] cells_in=3515, pos_cells=3150, density[min,max]=(0, 0.2401)
[zone 68] cells_in=3515, pos_cells=3497, density[min,max]=(0, 0.1102)
Готово: /root/globalmapper_learning/out_2/isolines.geojson | изолиний: 365
GRID=15.0 м, BANDWIDTH=10.0 м, уровни=[0.6, 0.75, 0.85, 0.92, 0.97], USE_E_WEIGHTS=True


In [270]:
# %% [markdown]
# Клетки под изолиниями + правило "3 стороны" + уровень кольца
# Входные пути берём как в вашем ноутбуке.

# %%
from __future__ import annotations
from pathlib import Path
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import Polygon, MultiPolygon, LineString, MultiLineString
from shapely.ops import unary_union

# --- ПАРАМЕТРЫ/ПУТИ ---
PATH_ZONES   = Path("/root/globalmapper_learning/zones.geojson")  # пока не используем
PATH_ISOLINE = Path("/root/globalmapper_learning/out_2/isolines.geojson")
PATH_GRID    = Path("/root/globalmapper_learning/out_2/buildings_clustered.geojson")

# В этой среде можно автоматически взять загруженные файлы
if Path("/mnt/data/isolines.geojson").exists():
    PATH_ISOLINE = Path("/mnt/data/isolines.geojson")
if Path("/mnt/data/buildings_clustered.geojson").exists():
    PATH_GRID    = Path("/mnt/data/buildings_clustered.geojson")

# Изолинии-ЛИНИИ буферим до полос (в метрах)
ISO_BUFFER_M = 1.0

# Порог для распознавания "сосед по стороне", доля от характерной длины ребра клетки
EDGE_SHARE_FRAC = 0.2  # 20% длины ребра (для 15 м → ~3 м)

OUT_TAGGED = PATH_GRID.with_name(f"{PATH_GRID.stem}_tagged_rings.geojson")

# --- ВСПОМОГАТЕЛЬНОЕ ---
def to_metric_crs(gdf: gpd.GeoDataFrame, fallback_epsg: int = 32636) -> gpd.GeoDataFrame:
    if gdf.crs is None:
        warnings.warn(f"CRS отсутствует, предполагаю EPSG:{fallback_epsg}")
        return gdf.set_crs(epsg=fallback_epsg, allow_override=True)
    if getattr(gdf.crs, "is_projected", None) is True:
        return gdf
    warnings.warn(f"CRS не метрический ({gdf.crs}), проецирую в EPSG:{fallback_epsg}")
    return gdf.to_crs(epsg=fallback_epsg)

def isolines_to_polys(iso_gdf: gpd.GeoDataFrame, buffer_m: float) -> gpd.GeoDataFrame:
    """Линии → полосы через buffer; полигоны оставляем как есть. Возвращает GeoDataFrame с индексом iso_pid."""
    iso_gdf = iso_gdf.copy()
    line_mask = iso_gdf.geom_type.isin(["LineString", "MultiLineString"])
    poly_mask = iso_gdf.geom_type.isin(["Polygon", "MultiPolygon"])

    parts = []
    if line_mask.any():
        lines = iso_gdf.loc[line_mask].copy()
        lines["geometry"] = lines.geometry.buffer(buffer_m)
        parts.append(lines)
    if poly_mask.any():
        parts.append(iso_gdf.loc[poly_mask].copy())

    out = pd.concat(parts, ignore_index=True) if parts else iso_gdf.copy()
    if not parts:
        out["geometry"] = out.geometry.buffer(buffer_m)

    out = gpd.GeoDataFrame(out, geometry="geometry", crs=iso_gdf.crs).reset_index(drop=True)
    out["iso_pid"] = np.arange(len(out))  # уникальный ID полигона изолинии (полосы)
    return out[["iso_pid", "geometry"]]

def rings_to_fill_polys(iso_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    «Половина изолинии с более высоким уровнем»: берём ВНУТРЕННОСТЬ замкнутых линий.
    - Для Polygon/MultiPolygon оставляем как есть (их внутренность уже задана).
    - Для LineString/MultiLineString создаём Polygon из замкнутых колец (is_ring=True).
    Возвращает GeoDataFrame с fill_id и geometry (могут пересекаться/вкладываться).
    """
    geoms = []
    for geom in iso_gdf.geometry:
        if geom is None or geom.is_empty:
            continue
        gtype = geom.geom_type
        if gtype in ("Polygon", "MultiPolygon"):
            if gtype == "Polygon":
                geoms.append(Polygon(geom.exterior))
            else:
                for p in geom.geoms:
                    geoms.append(Polygon(p.exterior))
        elif gtype in ("LineString", "MultiLineString"):
            if gtype == "LineString":
                if getattr(geom, "is_ring", False):
                    geoms.append(Polygon(geom.coords))
            else:
                for ls in geom.geoms:
                    if getattr(ls, "is_ring", False):
                        geoms.append(Polygon(ls.coords))
        # прочие типы игнорируем
    if not geoms:
        return gpd.GeoDataFrame({"fill_id": [], "geometry": []}, geometry="geometry", crs=iso_gdf.crs)

    # фиксим валидность тонким buffer(0)
    geoms = [g.buffer(0) if isinstance(g, (Polygon, MultiPolygon)) else g for g in geoms]
    fill = gpd.GeoDataFrame({"geometry": geoms}, geometry="geometry", crs=iso_gdf.crs)
    fill = fill[~fill.geometry.is_empty & fill.geometry.notna()].copy().reset_index(drop=True)
    fill["fill_id"] = np.arange(len(fill))
    return fill[["fill_id", "geometry"]]

def sjoin_intersects(left: gpd.GeoDataFrame, right: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    try:
        return gpd.sjoin(left, right, predicate="intersects", how="left")
    except TypeError:
        return gpd.sjoin(left, right, op="intersects", how="left")

def sjoin_within(left: gpd.GeoDataFrame, right: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    try:
        return gpd.sjoin(left, right, predicate="within", how="left")
    except TypeError:
        return gpd.sjoin(left, right, op="within", how="left")

# --- ЗАГРУЗКА ---
isoln = gpd.read_file(PATH_ISOLINE)
grid  = gpd.read_file(PATH_GRID)

# CRS → метрический
# grid  = to_metric_crs(grid)
isoln = isoln.to_crs(grid.crs)

# --- ИЗОЛИНИИ → ПОЛОСЫ (buffer) И УРОВНИ ВЛОЖЕННОСТИ ДЛЯ ПОЛОС ---
iso_polys = isolines_to_polys(isoln, buffer_m=ISO_BUFFER_M).copy()
iso_polys["iso_pid"] = pd.to_numeric(iso_polys["iso_pid"], errors="coerce").astype("Int64")
iso_polys = gpd.GeoDataFrame(iso_polys, geometry="geometry", crs=isoln.crs)

# representative_point каждого полигона-полосы → считаем, в скольких полосах он within; минус 1 (сам себя)
iso_pts = iso_polys[["iso_pid", "geometry"]].copy()
iso_pts["geometry"] = iso_pts.geometry.representative_point()
within_pairs = gpd.sjoin(
    iso_pts, iso_polys[["iso_pid", "geometry"]],
    how="left", predicate="within", lsuffix="pt", rsuffix="poly"
)
lvl = within_pairs.groupby("iso_pid_pt", dropna=False).size() - 1
iso_levels = (
    lvl.reset_index()
       .rename(columns={"iso_pid_pt": "iso_pid", 0: "iso_level"})
       .astype({"iso_pid": "Int64"})
)
iso_polys = iso_polys.merge(iso_levels, on="iso_pid", how="left")
iso_polys["iso_level"] = iso_polys["iso_level"].fillna(0).astype(int)

# --- НОВОЕ: ВНУТРЕННИЕ ПОЛИГОНЫ КОЛЕЦ (половина изолинии с более высоким уровнем) ---
iso_fill = rings_to_fill_polys(isoln)  # «внутренняя сторона» замкнутых линий

# Уровень вложенности для внутренних полигонов (по тому же принципу)
if len(iso_fill) > 0:
    fill_pts = iso_fill[["fill_id", "geometry"]].copy()
    fill_pts["geometry"] = fill_pts.geometry.representative_point()
    fill_pairs = gpd.sjoin(
        fill_pts, iso_fill[["fill_id", "geometry"]],
        how="left", predicate="within", lsuffix="pt", rsuffix="poly"
    )
    flvl = fill_pairs.groupby("fill_id_pt", dropna=False).size() - 1
    fill_levels = (
        flvl.reset_index()
            .rename(columns={"fill_id_pt": "fill_id", 0: "fill_level"})
            .astype({"fill_id": "Int64"})
    )
    iso_fill = iso_fill.merge(fill_levels, on="fill_id", how="left")
    iso_fill["fill_level"] = iso_fill["fill_level"].fillna(0).astype(int)

# --- ТЕГГИНГ КЛЕТОК ---
# 1) Пересечения с полосами (как было)
cells = grid[["geometry"]].reset_index(drop=False).rename(columns={"index": "cell_id"})
hit = gpd.sjoin(
    cells, iso_polys[["iso_pid", "iso_level", "geometry"]],
    how="left", predicate="intersects"
)
hit_nonnull = hit[hit["iso_pid"].notna()].copy()
agg_inter = (
    hit_nonnull.groupby("cell_id", dropna=False)
      .agg(
          iso_pids=("iso_pid", lambda s: sorted(set(int(x) for x in pd.to_numeric(s, errors="coerce").dropna().tolist()))),
          iso_level_raw=("iso_level", "max")
      )
      .reset_index()
)

grid = grid.reset_index(drop=False).rename(columns={"index": "cell_id"})
grid = grid.merge(agg_inter, on="cell_id", how="left")
grid["iso_pids"] = grid["iso_pids"].apply(lambda v: v if isinstance(v, list) else [])
grid["inside_iso_raw"] = grid["iso_pids"].apply(lambda v: len(v) > 0)
grid["iso_level_raw"] = pd.to_numeric(grid["iso_level_raw"], errors="coerce").astype("Float64")

# 2) ДОПОЛНИТЕЛЬНО: попадание центроидов внутрь «внутренних» полигонов колец
if len(iso_fill) > 0:
    cells_pts = cells.copy()
    # representative_point устойчивее centroid для невалидной геометрии
    cells_pts["geometry"] = grid.geometry.representative_point().values
    hit_fill = gpd.sjoin(
        cells_pts, iso_fill[["fill_id", "fill_level", "geometry"]],
        how="left", predicate="within"
    )
    hit_fill_nonnull = hit_fill[hit_fill["fill_id"].notna()].copy()
    agg_fill = (
        hit_fill_nonnull.groupby("cell_id", dropna=False)
            .agg(fill_level=("fill_level", "max"))
            .reset_index()
    )
    grid = grid.merge(agg_fill, on="cell_id", how="left")
    # клетки внутри внутренних полигонов тоже считаются raw-True
    inside_by_fill = grid["fill_level"].notna()
    grid.loc[inside_by_fill, "inside_iso_raw"] = True
    # уровень берём максимумом из прежнего и «внутреннего»
    grid["iso_level_raw"] = np.fmax(
        grid["iso_level_raw"].astype(float).fillna(-1),
        pd.to_numeric(grid["fill_level"], errors="coerce").fillna(-1)
    )
    grid["iso_level_raw"].replace(-1, np.nan, inplace=True)
else:
    grid["fill_level"] = np.nan

# --- ПРАВИЛО «3 СТОРОНЫ» ДЛЯ ПУСТЫХ КЛЕТОК (соседство по общему РЕБРУ) ---
pairs = gpd.sjoin(
    grid[["cell_id", "geometry"]], grid[["cell_id", "geometry"]],
    how="left", predicate="touches", lsuffix="a", rsuffix="b"
)
pairs = pairs[(pairs["cell_id_a"] != pairs["cell_id_b"]) & pairs["cell_id_b"].notna()].copy()
pairs["cell_id_b"] = pairs["cell_id_b"].astype(int)

# порог "существенного" ребра: >= 20% от характерной длины ребра клетки (~sqrt(area))
geom_list = list(grid.geometry.values)
pos = {rid: i for i, rid in enumerate(grid["cell_id"].values)}
edge_len_est = np.sqrt(np.maximum(grid.geometry.area.values, 1e-9))
thr_len = EDGE_SHARE_FRAC * edge_len_est

def _is_edge_neighbor(a: int, b: int) -> bool:
    ia, ib = pos[a], pos[b]
    inter = geom_list[ia].boundary.intersection(geom_list[ib].boundary)
    length = getattr(inter, "length", 0.0)
    return length >= min(thr_len[ia], thr_len[ib])

pairs["edge_ok"] = pairs.apply(lambda r: _is_edge_neighbor(int(r["cell_id_a"]), int(r["cell_id_b"])), axis=1)
pairs = pairs[pairs["edge_ok"]]

neighbors = {rid: [] for rid in grid["cell_id"].values}
for ra, rb in pairs[["cell_id_a", "cell_id_b"]].itertuples(index=False):
    neighbors[int(ra)].append(int(rb))

inside_map = dict(zip(grid["cell_id"].values, grid["inside_iso_raw"].values))
promote = []
for rid, neighs in neighbors.items():
    if not inside_map[rid]:
        if sum(1 for nb in neighs if inside_map.get(nb, False)) >= 3:
            promote.append(rid)

grid["inside_iso_closed"] = grid.apply(
    lambda r: bool(r["inside_iso_raw"] or (r["cell_id"] in promote)),
    axis=1
)

# --- УРОВЕНЬ ДЛЯ ДОЗАЖЖЁННЫХ КЛЕТОК — мода уровней соседей raw-True ---
level_map = dict(zip(grid["cell_id"].values, grid["iso_level_raw"].values))

def _neighbor_level_mode(rid: int) -> float:
    vals = [level_map.get(nb) for nb in neighbors.get(rid, []) if inside_map.get(nb, False) and pd.notna(level_map.get(nb))]
    if not vals:
        return np.nan
    return int(pd.Series(vals).value_counts().index[0])

grid["iso_level"] = grid["iso_level_raw"]
need_fill = grid["inside_iso_closed"] & (~grid["inside_iso_raw"])
grid.loc[need_fill, "iso_level"] = grid.loc[need_fill, "cell_id"].apply(_neighbor_level_mode)

# --- ВЫВОД ---
cols_out = ["cell_id", "geometry", "iso_pids", "inside_iso_raw", "inside_iso_closed", "iso_level_raw", "iso_level", "fill_level"]
grid_out = gpd.GeoDataFrame(grid[cols_out].copy(), geometry="geometry", crs=grid.crs)

OUT_TAGGED = PATH_GRID.with_name(f"{PATH_GRID.stem}_tagged_rings.geojson")
grid_out.to_file(OUT_TAGGED, driver="GeoJSON")
print(
    "OK →", OUT_TAGGED, "\n",
    f"Всего клеток: {len(grid_out)}, raw True: {int(grid_out.inside_iso_raw.sum())}, closed True: {int(grid_out.inside_iso_closed.sum())}"
)


/tmp/ipykernel_1800/2323380855.py:212: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  grid["iso_level_raw"].replace(-1, np.nan, inplace=True)


OK → /root/globalmapper_learning/out_2/buildings_clustered_tagged_rings.geojson 
 Всего клеток: 10065, raw True: 5439, closed True: 5613


In [5]:
# %% [markdown]
# Алгоритм размещения сервисов с участками:
# - Участки только прямоугольные, как набор клеток.
# - Корпус сервиса вписывается внутрь core-окна участка (внутренний буфер ≥1 клетка).
# - Зазоры: до домов ≥1 клетка (Chebyshev ≥2); между любыми участками ≥1 клетка (Chebyshev ≥2).
# - НОВОЕ: между участками ОДНОГО ТИПА ≥10 клеток (Chebyshev ≥10).
# - Размещение выполняется раунд-робином по типам сервисов: за один цикл ставим по одному объекту каждого типа.

from __future__ import annotations
from pathlib import Path
import math
import random
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
from shapely.ops import unary_union

# ---------- ПУТИ ----------
BASE_GRID = Path("/root/globalmapper_learning/out_2/buildings_clustered.geojson")
INPUT_CELLS_PATH = BASE_GRID.with_name(f"{BASE_GRID.stem}_tagged_rings.geojson")
PATH_ZONES = Path("/root/globalmapper_learning/zones.geojson")

# Если работаете в этой среде, можно подхватить из /mnt/data
if Path("/mnt/data/buildings_clustered_tagged_rings.geojson").exists():
    INPUT_CELLS_PATH = Path("/mnt/data/buildings_clustered_tagged_rings.geojson")
if Path("/mnt/data/zones.geojson").exists():
    PATH_ZONES = Path("/mnt/data/zones.geojson")

OUT_CELLS       = INPUT_CELLS_PATH.with_name(f"{INPUT_CELLS_PATH.stem.replace('_tagged_rings','')}_cells_with_building.geojson")
OUT_RECTS       = INPUT_CELLS_PATH.with_name(f"{INPUT_CELLS_PATH.stem.replace('_tagged_rings','')}_buildings_rects.geojson")
OUT_MERGED      = INPUT_CELLS_PATH.with_name(f"{INPUT_CELLS_PATH.stem.replace('_tagged_rings','')}_buildings_merged.geojson")
OUT_SVC_METRICS = INPUT_CELLS_PATH.with_name(f"{INPUT_CELLS_PATH.stem.replace('_tagged_rings','')}_service_uniformity.csv")
OUT_SVC_SITES   = INPUT_CELLS_PATH.with_name(f"{INPUT_CELLS_PATH.stem.replace('_tagged_rings','')}_service_sites.geojson")

# ---------- ПАРАМЕТРЫ ----------
MAX_RUN                = 8
NEIGH_EMPTY_THR        = 3
CELL_SIZE_M            = 15.0
EDGE_SHARE_FRAC        = 0.2
MERGE_PREDICATE        = "intersects"
MERGE_FIX_EPS          = 0.0

# Лимиты сервисов на квартал
MAX_SERVICES_PER_ZONE = {"school": 3, "kindergarten": 5, "polyclinics": 1}

# Рандомизация форм
RANDOMIZE_SERVICE_FORMS = True
SERVICE_RANDOM_SEED     = 42

# Зазоры
GAP_TO_HOUSES_CHEB      = 2  # территории держат ≥1 клетку до домов
GAP_BETWEEN_SITES_CHEB  = 2  # общая дистанция между участками (любыми типами)
SAME_TYPE_SITE_GAP_CHEB = 10 # НОВОЕ: между участками одного типа ≥10 клеток
INNER_MARGIN_CELLS      = 1  # корпус внутри территории с полем безопасности 1 клетка

# Нормативные площади участков (из таблицы сопоставлений)
SERVICE_SITE_RULES = {
    ("school", "RECT_5x2_WITH_OPEN_3"):      {"capacity": 600,  "site_area_m2": 33000.0},
    ("school", "H_5x4"):                      {"capacity": 800,  "site_area_m2": 36000.0},
    ("school", "RING_5x5_WITH_COURTYARD"):    {"capacity": 1100, "site_area_m2": 39600.0},
    ("kindergarten", "LINE3"):                {"capacity": 60,   "site_area_m2": 2640.0},
    ("kindergarten", "W5"):                   {"capacity": 100,  "site_area_m2": 4400.0},
    ("kindergarten", "H7"):                   {"capacity": 150,  "site_area_m2": 5700.0},
    ("polyclinics", "RECT_2x4"):              {"capacity": 300,  "site_area_m2": 3000.0},
}

# ---------------- УТИЛИТЫ ГЕО/ИНДЕКСАЦИЯ ----------------
def _min_site_cells_for_service_with_margin(svc: str,
                                            shape_variants: dict[str, list[tuple[str, list[list[tuple[int,int]]]]]],
                                            inner_margin_cells: int = 1) -> int:
    # минимальные клетки участка, чтобы любой вариант корпуса svc влез в core с отступом
    need = 0
    if svc not in shape_variants: 
        return 0
    best = None
    for _pat, vars_ in shape_variants[svc]:
        for var in vars_:
            vr = [dr for dr,_ in var]; vc = [dc for _,dc in var]
            h = (max(vr) - min(vr) + 1) + 2*inner_margin_cells
            w = (max(vc) - min(vc) + 1) + 2*inner_margin_cells
            cells_needed = h * w
            best = cells_needed if best is None else min(best, cells_needed)
    return 0 if best is None else best

def _sjoin_touches(a: gpd.GeoDataFrame, b: gpd.GeoDataFrame):
    try:
        return gpd.sjoin(a, b, how="left", predicate="touches", lsuffix="a", rsuffix="b")
    except TypeError:
        return gpd.sjoin(a, b, how="left", op="touches", lsuffix="a", rsuffix="b")

def _sjoin_generic(a: gpd.GeoDataFrame, b: gpd.GeoDataFrame, predicate: str):
    try:
        return gpd.sjoin(a, b, how="left", predicate=predicate, lsuffix="l", rsuffix="r")
    except TypeError:
        return gpd.sjoin(a, b, how="left", op=predicate, lsuffix="l", rsuffix="r")

def _grid_indices(gdf: gpd.GeoDataFrame):
    c = gdf.geometry.centroid
    x = c.x.values; y = c.y.values
    cell_step = np.median(np.sqrt(np.maximum(gdf.geometry.area.values, 1e-9)))
    x0, y0 = float(np.min(x)), float(np.min(y))
    col = np.rint((x - x0) / cell_step).astype(int)
    row = np.rint((y - y0) / cell_step).astype(int)
    return row, col, x0, y0, float(cell_step)

def _enforce_line_blocks(df, line_key, order_key, mask_key, max_run: int):
    out = df[mask_key].copy()
    for _, sub in df.loc[df[mask_key]].groupby(line_key):
        sub = sub.sort_values(order_key)
        idx = sub.index.to_numpy(); ordv = sub[order_key].to_numpy()
        breaks = np.where(np.diff(ordv) != 1)[0] + 1
        segments = np.split(np.arange(len(ordv)), breaks)
        for seg in segments:
            if len(seg) <= max_run: continue
            run = 0; place_gap = False
            for k in seg:
                i = idx[k]
                if place_gap:
                    out.loc[i] = False; place_gap = False; run = 0
                else:
                    if run < max_run:
                        out.loc[i] = True; run += 1
                        if run == max_run: place_gap = True
                    else:
                        out.loc[i] = False; run = 0
    return out

def _components(nodes: list[int], adj: dict[int, list[int]]) -> list[list[int]]:
    node_set = set(nodes); seen, comps = set(), []
    for v in nodes:
        if v in seen: continue
        stack, comp = [v], [v]; seen.add(v)
        while stack:
            u = stack.pop()
            for w in adj[u]:
                if w in node_set and w not in seen:
                    seen.add(w); stack.append(w); comp.append(w)
        comps.append(comp)
    return comps

def _pca_order(pts: np.ndarray) -> np.ndarray:
    if len(pts) <= 2: return np.argsort(pts[:,0] + pts[:,1])
    P = pts - pts.mean(axis=0, keepdims=True)
    _, _, Vt = np.linalg.svd(P, full_matrices=False); axis = Vt[0]
    t = P @ axis; return np.argsort(t)

def _make_valid(g):
    try: return g.buffer(0)
    except Exception: return g

def _compute_neighbors(cells: gpd.GeoDataFrame, edge_share_frac: float = 0.2):
    """
    Надёжная версия: вычисляет «смежность по стороне» на лету (без 'edge_ok').
    """
    left = cells[["geometry"]].reset_index().rename(columns={"index": "ida"})
    right = cells[["geometry"]].reset_index().rename(columns={"index": "idb"})
    pairs = _sjoin_touches(left, right)
    if "idb" not in pairs.columns and "index_right" in pairs.columns:
        pairs = pairs.rename(columns={"index_right": "idb"})
    pairs = pairs[(pairs["ida"] != pairs["idb"]) & pairs["idb"].notna()].copy()

    neighbors_all = {i: [] for i in range(len(cells))}
    neighbors_side = {i: [] for i in range(len(cells))}
    neighbors_diag = {i: [] for i in range(len(cells))}
    if len(pairs) > 0:
        pairs["idb"] = pairs["idb"].astype(int)
        geom_list = list(cells.geometry.values)
        edge_len_est = np.sqrt(np.maximum(cells.geometry.area.values, 1e-9))
        thr_len = edge_share_frac * edge_len_est

        def _is_edge_neighbor(a: int, b: int) -> bool:
            try:
                inter = geom_list[a].boundary.intersection(geom_list[b].boundary)
                length = getattr(inter, "length", 0.0)
            except Exception:
                length = 0.0
            return length >= min(thr_len[a], thr_len[b])

        for a, b in pairs[["ida", "idb"]].itertuples(index=False):
            a = int(a); b = int(b)
            eok = _is_edge_neighbor(a, b)
            neighbors_all[a].append(b)
            (neighbors_side if eok else neighbors_diag)[a].append(b)

        # симметрия
        for i in range(len(cells)):
            for dct in (neighbors_all, neighbors_side, neighbors_diag):
                for j in list(dct[i]):
                    if i not in dct[j]:
                        dct[j].append(i)

    inside = cells["inside_iso_closed"].fillna(False).to_numpy().astype(bool)
    empty_neighs = np.zeros(len(cells), dtype=int)
    missing = np.zeros(len(cells), dtype=int)
    for i in range(len(cells)):
        nn = list(set(neighbors_all.get(i, [])))
        empty_neighs[i] = sum(1 for j in nn if not inside[j])
        missing[i] = max(0, 8 - len(nn))

    return neighbors_all, neighbors_side, neighbors_diag, empty_neighs, missing

# ---------------- ПАТТЕРНЫ КОРПУСОВ СЕРВИСОВ ----------------
def _pattern_library():
    lib = {}
    # Kindergarten
    k_h7 = [(-1,-1),(0,-1),(1,-1),(0,0),(-1,1),(0,1),(1,1)]
    k_w5 = [(0,0),(1,1),(0,2),(1,3),(0,4)]
    line3 = [(0,0),(0,1),(0,2)]
    lib["kindergarten"] = [
        ("H7", k_h7, True),
        ("W5", k_w5, True),
        ("LINE3", line3, True),
    ]

    # Polyclinics
    rect_2x4 = [(r, c) for r in range(2) for c in range(4)]
    lib["polyclinics"] = [
        ("RECT_2x4", rect_2x4, True)
    ]

    # School
    s_h_5x4 = [(r,0) for r in range(5)] + [(r,3) for r in range(5)] + [(2,c) for c in range(4)]
    ring = []
    for r in range(5):
        for c in range(5):
            if (r in {0,4} or c in {0,4}) and not (r in {0,4} and c in {0,4}):
                ring.append((r,c))
    s_5x2_open = [(1,c) for c in range(5)] + [(0,0),(0,4)]
    lib["school"] = [
        ("H_5x4", s_h_5x4, True),
        ("RING_5x5_WITH_COURTYARD", ring, False),
        ("RECT_5x2_WITH_OPEN_3", s_5x2_open, True),
    ]
    return lib

def _transform_offsets(offsets, rot_k: int, mirror: bool):
    out = []
    for (dr, dc) in offsets:
        r, c = dr, dc
        for _ in range(rot_k % 4): r, c = c, -r
        if mirror: c = -c
        out.append((r, c))
    minr = min(r for r,_ in out); minc = min(c for _,c in out)
    return [(r - minr, c - minc) for (r,c) in out]

def _shape_variants(offsets, allow_rotations: bool):
    variants = set()
    rots = [0,1,2,3] if allow_rotations else [0]
    mirrors = [False, True] if allow_rotations else [False]
    for k in rots:
        for m in mirrors:
            var = tuple(sorted(_transform_offsets(offsets, k, m)))
            variants.add(var)
    return [list(v) for v in variants]

def _shape_length(var: list[tuple[int,int]]) -> int:
    rs = [dr for dr,_ in var]; cs = [dc for _,dc in var]
    return max(max(rs)-min(rs)+1, max(cs)-min(cs)+1)

# ---------------- ТЕРРИТОРИИ: ТОЛЬКО ПРЯМОУГОЛЬНИКИ ----------------
def _site_cells_required(area_m2: float, cell_size_m: float) -> int:
    return int(math.ceil(max(area_m2, 1.0) / (cell_size_m * cell_size_m)))

def _rect_variants_for_cells(ncells: int, max_variants: int = 12, ar_min: float = 0.33, ar_max: float = 3.0):
    base = int(round(math.sqrt(ncells)))
    pairs = []
    span = max(1, base) + 12
    for r in range(1, span+1):
        c = int(math.ceil(ncells / r))
        ar = max(r, c) / max(1.0, min(r, c))
        if ar_min <= ar <= ar_max:
            pairs.append((r, c, r*c - ncells))
    pairs = sorted(pairs, key=lambda t: (t[2], abs(t[0]-t[1])))
    pairs = pairs[:max_variants]
    variants = []
    for r, c, _ in pairs:
        offs = [(dr, dc) for dr in range(r) for dc in range(c)]
        variants.append((f"RECT_{r}x{c}", offs))
    return variants

def _territory_shape_variants(area_m2: float, cell_size_m: float) -> list[tuple[str, list[tuple[int,int]]]]:
    ncells = _site_cells_required(area_m2, cell_size_m)
    return _rect_variants_for_cells(ncells, max_variants=12)

# ---------------- ПРОЧЕЕ ----------------
def _centroid_of_indices(cells: gpd.GeoDataFrame, idxs: list[int]) -> Point:
    return unary_union([cells.geometry[i] for i in idxs]).representative_point()

def _min_cheb_between_sets(cells: gpd.GeoDataFrame, A: list[int], B: list[int]) -> int:
    if not A or not B: return 10**9
    ar = cells.loc[A, "row_i"].to_numpy(); ac = cells.loc[A, "col_j"].to_numpy()
    br = cells.loc[B, "row_i"].to_numpy(); bc = cells.loc[B, "col_j"].to_numpy()
    best = 10**9
    for i in range(len(ar)):
        dr = np.abs(br - ar[i]); dc = np.abs(bc - ac[i])
        d = int(np.min(np.maximum(dr, dc)))
        if d < best: best = d
        if best == 0: break
    return best

def _positions_from_center_or_edges(svc: str, rmin: int, rmax: int, cmin: int, cmax: int,
                                    r_center: float, c_center: float, invert: bool = False) -> list[tuple[int,int]]:
    pos = [(r0, c0) for r0 in range(rmin, rmax+1) for c0 in range(cmin, cmax+1)]
    def d2(rc): return (rc[0]-r_center)**2 + (rc[1]-c_center)**2
    pos.sort(key=d2, reverse=invert and (svc == "kindergarten"))
    return pos

# ---------- НОРМАТИВЫ → Параметры площадок ----------
def _service_site_spec(svc: str, pattern_name: str) -> tuple[float, int]:
    spec = SERVICE_SITE_RULES.get((svc, pattern_name))
    if spec:
        return float(spec["site_area_m2"]), int(spec["capacity"])
    if svc == "school":       return 33000.0, 600
    if svc == "kindergarten": return 4400.0, 100
    if svc == "polyclinics":  return 3000.0, 300
    return 2000.0, 0

# ------------------------------- ЗАГРУЗКА/ПОДГОТОВКА -------------------------------
cells = gpd.read_file(INPUT_CELLS_PATH).reset_index(drop=True)
if "inside_iso_closed" not in cells.columns:
    raise ValueError("Во входном слое отсутствует колонка 'inside_iso_closed'.")
cells["inside_iso_closed"] = cells["inside_iso_closed"].fillna(False).astype(bool)
cells["iso_level"] = pd.to_numeric(cells.get("iso_level"), errors="coerce")
cells["service"] = None  # корпуса сервисов будут помечаться позже

row_i, col_j, x0, y0, step_est = _grid_indices(cells)
cells["row_i"] = row_i; cells["col_j"] = col_j

zones = gpd.read_file(PATH_ZONES).to_crs(cells.crs).reset_index(drop=True)
zone_id_col = "zone_id"
if "id" in zones.columns: zones[zone_id_col] = zones["id"]
else: zones[zone_id_col] = np.arange(len(zones))
zone_name_col = "zone"
if zone_name_col not in zones.columns:
    for alt in ["functional_zone_type_name", "zone_type", "zone_name"]:
        if alt in zones.columns:
            zones[zone_name_col] = zones[alt]; break
if zone_name_col not in zones.columns: zones[zone_name_col] = "unknown"

cells_zone = gpd.sjoin(
    cells[["geometry"]].reset_index().rename(columns={"index":"cell_idx"}),
    zones[[zone_id_col, zone_name_col, "geometry"]],
    how="left", predicate="within"
).drop_duplicates("cell_idx")

cells = cells.merge(
    cells_zone[["cell_idx", zone_id_col, zone_name_col]],
    left_index=True, right_on="cell_idx", how="left"
).drop(columns=["cell_idx"])

cells[zone_name_col] = cells[zone_name_col].astype(str).str.lower().str.strip()
cells["is_residential_zone"] = cells[zone_name_col].eq("residential")

# Соседства/внешность
neighbors_all, neighbors_side, neighbors_diag, empty_neighs, missing = _compute_neighbors(
    cells, edge_share_frac=EDGE_SHARE_FRAC
)

# ---------------- 1) ЖИЛЫЕ ДОМА (фиксируем) ----------------
inside_mask = cells["inside_iso_closed"].values
is_external = np.zeros(len(cells), dtype=bool)
for i in range(len(cells)):
    if not inside_mask[i]: continue
    is_external[i] = (empty_neighs[i] >= NEIGH_EMPTY_THR) or (missing[i] > 0)
cells["candidate_building"] = inside_mask & is_external

cells["candidate_building"] = _enforce_line_blocks(cells, "row_i", "col_j", "candidate_building", MAX_RUN)
cells["is_building"] = _enforce_line_blocks(cells, "col_j", "row_i", "candidate_building", MAX_RUN)

# Промо внутренних соседей для «малых граничных»
bbox = np.array([cells.geometry.bounds.minx.values,
                 cells.geometry.bounds.miny.values,
                 cells.geometry.bounds.maxx.values,
                 cells.geometry.bounds.maxy.values]).T
w = bbox[:, 2] - bbox[:, 0]; h = bbox[:, 3] - bbox[:, 1]
is_small = (w < CELL_SIZE_M - 1e-6) | (h < CELL_SIZE_M - 1e-6)
external_score = empty_neighs + missing

is_b = cells["is_building"].to_numpy().astype(bool)
promote_targets = []
for i in range(len(cells)):
    if not (is_b[i] and is_external[i] and is_small[i]): continue
    cand = [j for j in neighbors_side.get(i, []) if inside_mask[j]]
    if not cand: cand = [j for j in neighbors_all.get(i, []) if inside_mask[j]]
    cand = [j for j in cand if not is_b[j]]
    if not cand: continue
    j_best = min(cand, key=lambda j: (external_score[j], -empty_neighs[j]))
    promote_targets.append(j_best)

if promote_targets:
    for j in promote_targets: is_b[j] = True
    cells["is_building"] = is_b
    cells["is_building"] = _enforce_line_blocks(cells, "row_i", "col_j", "is_building", MAX_RUN)
    cells["is_building"] = _enforce_line_blocks(cells, "col_j", "row_i", "is_building", MAX_RUN)

# ---------------- 2) ГРУППИРОВКА ПО КВАРТАЛАМ + ПОДГОТОВКА К ТЕРРИТОРИЯМ ----------------
idx_by_rc = {(int(r), int(c)): i for i,(r,c) in enumerate(zip(cells["row_i"], cells["col_j"]))}

is_res = cells["is_residential_zone"].fillna(False).values
not_house = ~cells["is_building"].fillna(False).values
inside_true  = cells["inside_iso_closed"].fillna(False).values
inside_false = ~inside_true
ok_iso = (cells["iso_level"].fillna(0).values >= 0)

zone_groups: dict[int, dict] = {}
for zid, sub_all in cells[pd.notna(cells["zone_id"])].groupby(cells["zone_id"].astype("Int64")):
    zid = int(zid)
    in_ids = sub_all.index[(is_res[sub_all.index]) & (inside_true[sub_all.index]) & ok_iso[sub_all.index] & not_house[sub_all.index]].to_list()
    out_ids = sub_all.index[(is_res[sub_all.index]) & (inside_false[sub_all.index]) & not_house[sub_all.index]].to_list()
    if not in_ids and not out_ids:
        continue
    sub_res = sub_all.index[is_res[sub_all.index]].to_list()
    r_center = float(cells.loc[sub_res, "row_i"].median()) if len(sub_res) else float(sub_all["row_i"].median())
    c_center = float(cells.loc[sub_res, "col_j"].median()) if len(sub_res) else float(sub_all["col_j"].median())
    Lmax = int(pd.to_numeric(cells.loc[in_ids, "iso_level"], errors="coerce").fillna(0).max()) if in_ids else 0
    zone_groups[zid] = {
        "inside_ids": in_ids,
        "outside_ids": out_ids,
        "r_center": r_center,
        "c_center": c_center,
        "Lmax": Lmax,
    }

svc_count_by_zone = {int(z): {"school":0,"kindergarten":0,"polyclinics":0} for z in zone_groups.keys()}
rng = random.Random(SERVICE_RANDOM_SEED)

lib = _pattern_library()
shape_variants = {svc: [] for svc in lib.keys()}
for svc, shapes in lib.items():
    items = list(shapes)
    if RANDOMIZE_SERVICE_FORMS: rng.shuffle(items)
    for name, offsets, allow_rot in items:
        vars_ = _shape_variants(offsets, allow_rot)
        if RANDOMIZE_SERVICE_FORMS: rng.shuffle(vars_)
        shape_variants[svc].append((name, vars_))

reserved_site_cells: set[int] = set()     # клетки — территории
reserved_service_cells: set[int] = set()  # клетки — корпуса (подмножество соответствующей территории)

service_sites_geom = []
service_sites_attrs = []
service_polys_geom = []
service_polys_attrs = []

# ===== ЧАСТЬ 2/2 =====

# ---------- ПРОВЕРКИ ЗАЗОРОВ ----------
def _house_indices() -> np.ndarray:
    return np.where(cells["is_building"].fillna(False).to_numpy())[0]

def _cheb_gap_ok_to_houses(candidate_idxs: list[int]) -> bool:
    H = _house_indices()
    if len(H) == 0 or len(candidate_idxs) == 0:
        return True
    d = _min_cheb_between_sets(cells, candidate_idxs, list(H))
    return d >= GAP_TO_HOUSES_CHEB

def _cheb_gap_ok_to_sites(candidate_idxs: list[int],
                          placed_site_sets: list[list[int]],
                          placed_sites_by_type: dict[str, list[list[int]]],
                          svc: str) -> bool:
    # Общий зазор со всеми участками (любой тип)
    for S in placed_site_sets:
        if _min_cheb_between_sets(cells, candidate_idxs, S) < GAP_BETWEEN_SITES_CHEB:
            return False
    # Усиленный зазор для участков того же типа
    for S in placed_sites_by_type.get(svc, []):
        if _min_cheb_between_sets(cells, candidate_idxs, S) < SAME_TYPE_SITE_GAP_CHEB:
            return False
    return True

# ---------- ВСПОМОГАТЕЛЬНО: сортировка паттернов под ориентацию core-окна ----------
def _sort_variants_by_core_fit(variants: list[list[tuple[int,int]]], core_h: int, core_w: int) -> list[list[tuple[int,int]]]:
    if core_h <= 0 or core_w <= 0:
        return []
    core_ar = core_w / core_h if core_h > 0 else 1.0
    def dims(var):
        vr = [dr for (dr,dc) in var]; vc = [dc for (dr,dc) in var]
        h = max(vr) - min(vr) + 1
        w = max(vc) - min(vc) + 1
        return h, w
    scored = []
    for var in variants:
        h, w = dims(var)
        var_ar = w / h if h > 0 else 1.0
        same_orient = int(not ((core_w >= core_h) ^ (w >= h)))  # 1 если ориентации совпали
        ar_diff = abs(math.log(max(var_ar,1e-6)/max(core_ar,1e-6)))
        scored.append((0 if same_orient else 1, ar_diff, h*w, var))
    scored.sort(key=lambda t: (t[0], t[1], t[2]))
    return [t[3] for t in scored]

# ---------- КОРПУС ВНУТРИ ТЕРРИТОРИИ (core >= 1 клетка от границ) ----------
def _try_place_service_inside_site(svc: str, zid: int, site_idxs: list[int],
                                   site_id: str,
                                   shape_variants: dict[str, list[tuple[str, list[list[tuple[int,int]]]]]],
                                   reserved_service_cells: set[int],
                                   rng: random.Random,
                                   inner_margin_cells: int = INNER_MARGIN_CELLS) -> tuple[bool, list[int], str]:
    site_set = set(site_idxs)
    rvals = cells.loc[site_idxs, "row_i"].to_numpy()
    cvals = cells.loc[site_idxs, "col_j"].to_numpy()
    rmin, rmax = int(rvals.min()), int(rvals.max())
    cmin, cmax = int(cvals.min()), int(cvals.max())

    # Core-окно (эрозия на inner_margin_cells)
    rmin_core = rmin + inner_margin_cells
    rmax_core = rmax - inner_margin_cells
    cmin_core = cmin + inner_margin_cells
    cmax_core = cmax - inner_margin_cells
    if (rmin_core > rmax_core) or (cmin_core > cmax_core):
        return False, [], ""

    core_h = rmax_core - rmin_core + 1
    core_w = cmax_core - cmin_core + 1
    cen_r = 0.5 * (rmin_core + rmax_core)
    cen_c = 0.5 * (cmin_core + cmax_core)

    for (pat_name, variants) in shape_variants.get(svc, []):
        vars_iter = list(variants)
        if RANDOMIZE_SERVICE_FORMS:
            rng.shuffle(vars_iter)
        vars_iter = _sort_variants_by_core_fit(vars_iter, core_h, core_w)

        for var in vars_iter:
            vr = [dr for (dr,dc) in var]; vc = [dc for (dr,dc) in var]
            h, w = (max(vr)-min(vr)+1), (max(vc)-min(vc)+1)
            if h > core_h or w > core_w:
                continue

            positions = [(r0, c0)
                         for r0 in range(rmin_core, rmax_core - h + 2)
                         for c0 in range(cmin_core, cmax_core - w + 2)]
            positions.sort(key=lambda rc: (rc[0]-cen_r)**2 + (rc[1]-cen_c)**2)

            for (r0, c0) in positions:
                idxs = []
                ok = True
                for (dr, dc) in var:
                    rr, cc = r0 + dr, c0 + dc
                    idx = idx_by_rc.get((rr, cc))
                    if (idx is None) or (idx in reserved_service_cells) or (idx not in site_set):
                        ok = False; break
                    idxs.append(idx)
                if not ok:
                    continue
                return True, idxs, pat_name

    return False, [], ""

# ---------- ТЕРРИТОРИЯ + КОРПУС (внутри core) ДЛЯ iso>=L ----------
def _try_place_site_and_service_in_zone_level(zid: int, svc: str, allowed_ids: list[int],
                                              r_cen: float, c_cen: float,
                                              placed_site_sets: list[list[int]],
                                              placed_sites_by_type: dict[str, list[list[int]]],
                                              rng: random.Random) -> bool:
    if not allowed_ids:
        return False

    allowed_set = set(allowed_ids)
    coord_to_idx = {(int(cells.at[i,"row_i"]), int(cells.at[i,"col_j"])): i for i in allowed_ids}
    sub = cells.loc[allowed_ids, ["row_i","col_j"]]
    rmin, rmax = int(sub["row_i"].min()), int(sub["row_i"].max())
    cmin, cmax = int(sub["col_j"].min()), int(sub["col_j"].max())

    positions = _positions_from_center_or_edges(svc, rmin, rmax, cmin, cmax, r_cen, c_cen, invert=False)

    service_variants = list(shape_variants.get(svc, []))
    if RANDOMIZE_SERVICE_FORMS:
        rng.shuffle(service_variants)

    for (pat_name, _service_vars) in service_variants:
        site_area_m2, capacity = _service_site_spec(svc, pat_name)
        territory_variants = _territory_shape_variants(site_area_m2, CELL_SIZE_M)
        if RANDOMIZE_SERVICE_FORMS:
            rng.shuffle(territory_variants)

        for (site_form_name, site_offsets) in territory_variants:
            vrr = [dr for (dr,dc) in site_offsets]; vcc = [dc for (dr,dc) in site_offsets]
            Hs, Ws = (max(vrr)-min(vrr)+1), (max(vcc)-min(vcc)+1)

            for (r0, c0) in positions:
                if r0 + Hs - 1 > rmax or c0 + Ws - 1 > cmax:
                    continue

                site_idxs = []
                ok = True
                for (dr, dc) in site_offsets:
                    rr, cc = r0 + dr, c0 + dc
                    idx = coord_to_idx.get((rr, cc))
                    if (idx is None) or (idx in reserved_site_cells) or (idx in reserved_service_cells):
                        ok = False; break
                    if idx not in allowed_set:
                        ok = False; break
                    if cells.at[idx, "is_building"]:
                        ok = False; break
                    site_idxs.append(idx)
                if not ok:
                    continue

                # зазоры: до домов, до любых участков, и усиленный до участков того же типа
                if not _cheb_gap_ok_to_houses(site_idxs):
                    continue
                if not _cheb_gap_ok_to_sites(site_idxs, placed_site_sets, placed_sites_by_type, svc):
                    continue

                # корпус внутри core-окна участка
                ok_svc, svc_cell_idxs, chosen_pat = _try_place_service_inside_site(
                    svc, zid, site_idxs, site_id="__tmp__",
                    shape_variants=shape_variants,
                    reserved_service_cells=reserved_service_cells,
                    rng=rng,
                    inner_margin_cells=INNER_MARGIN_CELLS
                )
                if not ok_svc:
                    continue

                # --- УСПЕХ: фиксируем участок и корпус ---
                site_id = f"SITE_{svc.upper()}_{str(len(service_sites_attrs)+1).zfill(4)}"
                service_id = f"{svc.upper()}_{str(len(service_polys_attrs)+1).zfill(4)}"

                for idx in site_idxs:
                    reserved_site_cells.add(idx)
                    cells.loc[idx, "is_service_site"] = True
                    cells.loc[idx, "site_id"] = site_id
                    cells.loc[idx, "service_site_type"] = svc

                for idx in svc_cell_idxs:
                    reserved_service_cells.add(idx)
                    cells.loc[idx, "service"] = svc

                site_poly = _make_valid(unary_union([cells.geometry[i] for i in site_idxs]))
                svc_poly  = _make_valid(unary_union([cells.geometry[i] for i in svc_cell_idxs]))

                service_sites_geom.append(site_poly)
                service_sites_attrs.append({
                    "site_id": site_id,
                    "service": svc,
                    "zone_id": int(zid),
                    "site_form": site_form_name,
                    "pattern_for_norms": pat_name,
                    "site_cells": int(len(site_idxs)),
                    "site_area_target_m2": float(site_area_m2),
                    "site_area_actual_m2": float(getattr(site_poly, "area", 0.0)),
                    "capacity": int(capacity),
                })

                service_polys_geom.append(svc_poly)
                service_polys_attrs.append({
                    "building_id": service_id,
                    "site_id": site_id,
                    "service": svc,
                    "pattern": chosen_pat,
                    "zone_id": int(zid),
                    "n_cells": int(len(svc_cell_idxs)),
                    "width_m": float(CELL_SIZE_M),
                })

                placed_site_sets.append(site_idxs)
                placed_sites_by_type.setdefault(svc, []).append(site_idxs)
                svc_count_by_zone[zid][svc] = svc_count_by_zone.get(zid, {svc:0}).get(svc,0) + 1
                return True

    return False

# ---------- FALLBACK: ВНЕ inside_iso ----------
def _try_place_site_and_service_fallback_outside(zid: int, svc: str, outside_ids: list[int],
                                                 r_cen: float, c_cen: float,
                                                 placed_site_sets: list[list[int]],
                                                 placed_sites_by_type: dict[str, list[list[int]]],
                                                 rng: random.Random) -> bool:
    if not outside_ids:
        return False

    allowed_ids = [i for i in outside_ids if (i not in reserved_site_cells) and (not cells.at[i,"is_building"])]
    if not allowed_ids:
        return False

    allowed_set = set(allowed_ids)
    coord_to_idx = {(int(cells.at[i,"row_i"]), int(cells.at[i,"col_j"])): i for i in allowed_ids}
    sub = cells.loc[allowed_ids, ["row_i","col_j"]]
    rmin, rmax = int(sub["row_i"].min()), int(sub["row_i"].max())
    cmin, cmax = int(sub["col_j"].min()), int(sub["col_j"].max())
    positions = _positions_from_center_or_edges(svc, rmin, rmax, cmin, cmax, r_cen, c_cen, invert=False)

    service_variants = list(shape_variants.get(svc, []))
    if RANDOMIZE_SERVICE_FORMS:
        rng.shuffle(service_variants)

    for (pat_name, _service_vars) in service_variants:
        site_area_m2, capacity = _service_site_spec(svc, pat_name)
        min_cells = _min_site_cells_for_service_with_margin(svc, shape_variants, INNER_MARGIN_CELLS)
        ncells = max(_site_cells_required(site_area_m2, CELL_SIZE_M), min_cells)
        territory_variants = _rect_variants_for_cells(ncells, max_variants=12)
        if RANDOMIZE_SERVICE_FORMS:
            rng.shuffle(territory_variants)

        for (site_form_name, site_offsets) in territory_variants:
            vrr = [dr for (dr,dc) in site_offsets]; vcc = [dc for (dr,dc) in site_offsets]
            Hs, Ws = (max(vrr)-min(vrr)+1), (max(vcc)-min(vcc)+1)

            for (r0, c0) in positions:
                if r0 + Hs - 1 > rmax or c0 + Ws - 1 > cmax:
                    continue

                site_idxs = []
                ok = True
                for (dr, dc) in site_offsets:
                    rr, cc = r0 + dr, c0 + dc
                    idx = coord_to_idx.get((rr, cc))
                    if (idx is None) or (idx in reserved_site_cells) or (idx in reserved_service_cells):
                        ok = False; break
                    if idx not in allowed_set:
                        ok = False; break
                    if cells.at[idx, "is_building"]:
                        ok = False; break
                    site_idxs.append(idx)
                if not ok:
                    continue

                if not _cheb_gap_ok_to_houses(site_idxs):
                    continue
                if not _cheb_gap_ok_to_sites(site_idxs, placed_site_sets, placed_sites_by_type, svc):
                    continue

                ok_svc, svc_cell_idxs, chosen_pat = _try_place_service_inside_site(
                    svc, zid, site_idxs, site_id="__tmp__",
                    shape_variants=shape_variants,
                    reserved_service_cells=reserved_service_cells,
                    rng=rng,
                    inner_margin_cells=INNER_MARGIN_CELLS
                )
                if not ok_svc:
                    continue

                # --- УСПЕХ: фиксируем участок и корпус ---
                site_id = f"SITE_{svc.upper()}_{str(len(service_sites_attrs)+1).zfill(4)}"
                service_id = f"{svc.upper()}_{str(len(service_polys_attrs)+1).zfill(4)}"

                for idx in site_idxs:
                    reserved_site_cells.add(idx)
                    cells.loc[idx, "is_service_site"] = True
                    cells.loc[idx, "site_id"] = site_id
                    cells.loc[idx, "service_site_type"] = svc

                for idx in svc_cell_idxs:
                    reserved_service_cells.add(idx)
                    cells.loc[idx, "service"] = svc

                site_poly = _make_valid(unary_union([cells.geometry[i] for i in site_idxs]))
                svc_poly  = _make_valid(unary_union([cells.geometry[i] for i in svc_cell_idxs]))

                service_sites_geom.append(site_poly)
                service_sites_attrs.append({
                    "site_id": site_id,
                    "service": svc,
                    "zone_id": int(zid),
                    "site_form": site_form_name,
                    "pattern_for_norms": pat_name,
                    "site_cells": int(len(site_idxs)),
                    "site_area_target_m2": float(site_area_m2),
                    "site_area_actual_m2": float(getattr(site_poly, "area", 0.0)),
                    "capacity": int(capacity),
                    "fallback_outside": True,
                })

                service_polys_geom.append(svc_poly)
                service_polys_attrs.append({
                    "building_id": service_id,
                    "site_id": site_id,
                    "service": svc,
                    "pattern": chosen_pat,
                    "zone_id": int(zid),
                    "n_cells": int(len(svc_cell_idxs)),
                    "width_m": float(CELL_SIZE_M),
                    "fallback_outside": True,
                })

                placed_site_sets.append(site_idxs)
                placed_sites_by_type.setdefault(svc, []).append(site_idxs)
                svc_count_by_zone[zid][svc] = svc_count_by_zone.get(zid, {svc:0}).get(svc,0) + 1
                return True

    return False

# ---------- ОСНОВНОЙ ЦИКЛ: РАУНД-РОБИН ПО ТИПАМ (по одному за проход) ----------
placed_site_sets: list[list[int]] = []                         # глобальный список всех участков (для общего зазора)
svc_order = ["school", "kindergarten", "polyclinics"]
placed_sites_by_type: dict[str, list[list[int]]] = {s: [] for s in svc_order}  # НОВОЕ: для усиленного зазора 10 клеток

for zid, meta in zone_groups.items():
    r_cen, c_cen = meta["r_center"], meta["c_center"]
    Lmax = meta["Lmax"]
    z_inside = meta["inside_ids"]
    z_outside = meta["outside_ids"]

    limits = {svc: MAX_SERVICES_PER_ZONE.get(svc, 0) for svc in svc_order}

    # Крутимся, пока в этом квартале удаётся что-то поставить за один полный проход
    while True:
        progress = False
        for svc in svc_order:
            if svc_count_by_zone[zid][svc] >= limits[svc]:
                continue

            # Пытаемся поставить ровно ОДИН объект этого svc за цикл:
            placed_here = False
            for L in range(Lmax, -1, -1):
                allowed_ids = [i for i in z_inside
                               if (i not in reserved_site_cells)
                               and (not cells.at[i,"is_building"])
                               and pd.notna(cells.at[i, "iso_level"])
                               and (int(cells.at[i,"iso_level"]) >= L)]
                if not allowed_ids:
                    continue
                if _try_place_site_and_service_in_zone_level(
                        zid, svc, allowed_ids, r_cen, c_cen,
                        placed_site_sets, placed_sites_by_type, rng):
                    placed_here = True
                    break
            if not placed_here and z_outside:
                if _try_place_site_and_service_fallback_outside(
                        zid, svc, z_outside, r_cen, c_cen,
                        placed_site_sets, placed_sites_by_type, rng):
                    placed_here = True

            if placed_here:
                progress = True  # За этот проход мы поставили ОДИН объект данного типа
            # Идём к следующему типу, даже если можно было бы поставить ещё — «по одному за цикл»

        if not progress:
            break  # В этом квартале больше ничего не вставляется с учётом всех ограничений

# ---------- СБОРКА ЖИЛЫХ ПОЛИГОНОВ ----------
is_b = cells["is_building"].to_numpy().astype(bool)

diag_only_nodes = []
for i in range(len(cells)):
    if not is_b[i]: continue
    side_in = any(is_b[j] for j in neighbors_side.get(i, []))
    diag_in = any(is_b[j] for j in neighbors_diag.get(i, []))
    if (not side_in) and diag_in:
        diag_only_nodes.append(i)

diag_adj = {i: [j for j in neighbors_diag.get(i, []) if is_b[j]] for i in range(len(cells))}
diag_components = _components(diag_only_nodes, diag_adj)
diag_components = [comp for comp in diag_components if len(comp) >= 2]

cells["is_diag_only"] = False
for comp in diag_components:
    for i in comp: cells.at[i, "is_diag_only"] = True

records_geom, records_attrs = [], []

try:
    from shapely.geometry import CAP_STYLE, JOIN_STYLE
    cap_style = CAP_STYLE.flat; join_style = JOIN_STYLE.mitre
except Exception:
    cap_style = 2; join_style = 2

buf_dist = float(CELL_SIZE_M) / 2.0
centroids = cells.geometry.centroid

# Диагональные жилые → полоса-буфер
for k, comp in enumerate(diag_components, start=1):
    pts = np.array([[centroids[i].x, centroids[i].y] for i in comp], dtype=float)
    if len(pts) < 2: continue
    order = _pca_order(pts)
    ordered_pts = pts[order]
    ordered_pts = ordered_pts[np.concatenate(([True], np.any(np.diff(ordered_pts, axis=0) != 0, axis=1)))]
    if len(ordered_pts) < 2: continue
    line = LineString(ordered_pts)
    poly = line.buffer(buf_dist, cap_style=cap_style, join_style=join_style)
    records_geom.append(_make_valid(poly))
    records_attrs.append({
        "building_id": f"D{str(k).zfill(5)}",
        "type": "diag_buffer",
        "service": "living_house",
        "n_cells": int(len(comp)),
        "width_m": float(CELL_SIZE_M),
    })

# Простые жилые клетки
simple_ids = [i for i in range(len(cells))
              if cells.at[i,"is_building"] and not cells.at[i,"is_diag_only"]]
for i in simple_ids:
    records_geom.append(_make_valid(cells.geometry[i]))
    records_attrs.append({
        "building_id": f"C{str(i).zfill(6)}",
        "type": "cell",
        "service": "living_house",
        "n_cells": 1,
        "width_m": float(CELL_SIZE_M),
        "row_i": int(cells.at[i, "row_i"]),
        "col_j": int(cells.at[i, "col_j"]),
        zone_id_col: int(cells.at[i, zone_id_col]) if not pd.isna(cells.at[i, zone_id_col]) else None,
    })

# Добавляем корпуса сервисов
records_geom.extend(service_polys_geom)
records_attrs.extend(service_polys_attrs)

buildings_rects = gpd.GeoDataFrame(records_attrs, geometry=records_geom, crs=cells.crs).reset_index(drop=True)

# ---------- MERGE без склейки разных service ----------
left = buildings_rects[["geometry"]].reset_index().rename(columns={"index": "i"})
right = buildings_rects[["geometry"]].reset_index().rename(columns={"index": "j"})
pairs = _sjoin_generic(left, right, predicate=MERGE_PREDICATE)
pairs = pairs[(pairs["i"] != pairs["j"]) & pairs["j"].notna()].copy()
pairs["j"] = pairs["j"].astype(int)

svc_vals = buildings_rects.get("service", pd.Series(["living_house"]*len(buildings_rects))).astype(object).tolist()
def _same_group(i,j):
    si, sj = svc_vals[i], svc_vals[j]
    return si == sj

pairs = pairs[pairs.apply(lambda r: _same_group(int(r["i"]), int(r["j"])), axis=1)]

adj = {i: [] for i in range(len(buildings_rects))}
for a, b in pairs[["i", "j"]].itertuples(index=False):
    a = int(a); b = int(b)
    adj[a].append(b); adj[b].append(a)

groups = _components(list(adj.keys()), adj)

merged_geoms = []; merged_attrs = []
for gid, comp in enumerate(groups):
    geoms = buildings_rects.geometry.iloc[comp].tolist()
    if MERGE_FIX_EPS and MERGE_FIX_EPS > 0:
        u = unary_union([_make_valid(g.buffer(MERGE_FIX_EPS)) for g in geoms]).buffer(-MERGE_FIX_EPS)
    else:
        u = unary_union([_make_valid(g) for g in geoms])
    comp_svc = list({svc_vals[i] for i in comp})
    merged_service = comp_svc[0] if len(comp_svc) == 1 else "mixed"
    types = ",".join(sorted(set(buildings_rects.loc[comp, "type"].astype(str).tolist())))
    zid_mode = None
    if zone_id_col in buildings_rects.columns:
        zids = buildings_rects.loc[comp, zone_id_col].dropna().astype(int)
        if len(zids) > 0: zid_mode = int(zids.value_counts().index[0])
    merged_geoms.append(_make_valid(u))
    merged_attrs.append({
        "group_id": int(gid),
        "n_members": int(len(comp)),
        "n_cells_sum": int(np.nansum(buildings_rects.loc[comp, "n_cells"].values)) if "n_cells" in buildings_rects else None,
        "types": types,
        "service": merged_service,
        zone_id_col: zid_mode,
    })

buildings_merged = gpd.GeoDataFrame(merged_attrs, geometry=merged_geoms, crs=cells.crs).reset_index(drop=True)
buildings_merged = buildings_merged[buildings_merged.area > CELL_SIZE_M*CELL_SIZE_M]

# ---------- МЕТРИКИ (по участкам сервисов) ----------
service_sites_gdf = gpd.GeoDataFrame(service_sites_attrs, geometry=service_sites_geom, crs=cells.crs).reset_index(drop=True)

metrics_rows = []
if len(service_sites_gdf) > 0:
    service_sites_gdf["centroid"] = service_sites_gdf.geometry.representative_point()
    for (zid, svc), sub in service_sites_gdf.groupby([zone_id_col, "service"], dropna=True):
        pts = list(sub["centroid"])
        n = len(pts)
        if n == 1:
            min_nn = float("inf"); mean_nn = float("inf"); score = 1.0
        else:
            dmat = np.zeros((n, n), dtype=float)
            for i in range(n):
                for j in range(i+1, n):
                    d = pts[i].distance(pts[j])
                    dmat[i,j] = dmat[j,i] = d
            nn = np.min(np.where(dmat==0, np.inf, dmat), axis=1)
            min_nn = float(np.min(nn)); mean_nn = float(np.mean(nn))
            score = float(mean_nn / (CELL_SIZE_M))
        metrics_rows.append({
            zone_id_col: int(zid),
            "service": svc,
            "sites_count": int(n),
            "min_nn_m": None if math.isinf(min_nn) else float(min_nn),
            "mean_nn_m": None if math.isinf(mean_nn) else float(mean_nn),
            "uniformity_score": score,
        })
metrics_df = pd.DataFrame(metrics_rows)
if len(metrics_df) > 0:
    metrics_df.to_csv(OUT_SVC_METRICS, index=False)

# ---------- Финальные статусы в клетках ----------
cells.loc[cells["is_building"].fillna(False), "service"] = cells.loc[cells["is_building"].fillna(False), "service"].fillna("living_house")

# ---------- Сохранение ----------
cells_out = cells.copy()
cells_out.to_file(OUT_CELLS, driver="GeoJSON")
buildings_rects.to_file(OUT_RECTS, driver="GeoJSON")
buildings_merged.to_file(OUT_MERGED, driver="GeoJSON")
if len(service_sites_gdf) > 0:
    service_sites_gdf.drop(columns=['centroid'], inplace=True)
    service_sites_gdf.to_file(OUT_SVC_SITES, driver="GeoJSON")

svc_rects = buildings_rects[buildings_rects.get("service").isin(["school","kindergarten","polyclinics"])]

print(
    f"OK → {OUT_CELLS.name}  (клеток: {len(cells_out)}, жилых: {int(cells_out['is_building'].sum())}, diag-only: {int(cells_out['is_diag_only'].sum())})\n"
    f"OK → {OUT_RECTS.name}  (элементов: {len(buildings_rects)}, сервис-корпусов: {len(svc_rects)})\n"
    f"OK → {OUT_MERGED.name} (полигонов: {len(buildings_merged)}, "
    f"schools: {int((buildings_merged.get('service')=='school').sum())}, "
    f"kindergartens: {int((buildings_merged.get('service')=='kindergarten').sum())}, "
    f"polyclinics: {int((buildings_merged.get('service')=='polyclinics').sum())}, "
    f"living_house: {int((buildings_merged.get('service')=='living_house').sum())})\n"
    f"Uniformity CSV: {OUT_SVC_METRICS.name} (по УЧАСТКАМ)\n"
    f"Service sites: {OUT_SVC_SITES.name} (участков: {len(service_sites_gdf)})"
)


OK → buildings_clustered_cells_with_building.geojson  (клеток: 10065, жилых: 1463, diag-only: 69)
OK → buildings_clustered_buildings_rects.geojson  (элементов: 1430, сервис-корпусов: 7)
OK → buildings_clustered_buildings_merged.geojson (полигонов: 253, schools: 1, kindergartens: 5, polyclinics: 1, living_house: 246)
Uniformity CSV: buildings_clustered_service_uniformity.csv (по УЧАСТКАМ)
Service sites: buildings_clustered_service_sites.geojson (участков: 7)


In [6]:
# %% [markdown]
# Расчёт этажности и распределение жилой площади по жилым домам
# Правила:
# 1) floors_count = round(area_m2 / 100), но: floors_count ∈ [1, max_floor(zone)]
# 2) Жилая площадь в доме: living_area = K * area_m2 * floors_count, где K ∈ [0.5, 0.75]
# 3) Целевую площадь target_living_area по зоне распределяем с приоритетом к границам квартала
#    (вес = 1 / distance_to_zone_boundary), с соблюдением K-ограничений.
# ДОПОЛНИТЕЛЬНО:
# 4) Жилая площадь вычисляется ТОЛЬКО для зон, где явно задан target_living_area; для остальных living_area=0, K=NaN.
# 5) Этажность сервисов фиксирована: kindergarten=2, school=3, polyclinics=4 (и не превышает max_floor).

from __future__ import annotations
from pathlib import Path
import math
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.ops import unary_union

# ---------------- ПОЛЬЗОВАТЕЛЬСКИЕ ВХОДЫ ----------------
BUILDINGS_PATH = Path("/root/globalmapper_learning/out_2/buildings_clustered_buildings_merged.geojson")
ZONES_PATH     = Path("/root/globalmapper_learning/zones.geojson")

# Автоподхват из /mnt/data при наличии
if Path("/mnt/data/buildings_merged.geojson").exists():
    BUILDINGS_PATH = Path("/mnt/data/buildings_merged.geojson")
if Path("/mnt/data/zones.geojson").exists():
    ZONES_PATH = Path("/mnt/data/zones.geojson")

# Параметры по зонам:
# Вариант А: по zone_id
ZONE_PARAMS_BY_ID = {
    # пример: 101: {"target_living_area": 120000.0, "max_floor": 12},
}
# Вариант Б: по названию зоны (поле "zone" или аналог)
ZONE_PARAMS_BY_NAME = {
    "residential": {"target_living_area": 20000.0, "max_floor": 9},
    "industrial":  {"target_living_area": 0.0,     "max_floor": 5},
    "business":    {"target_living_area": 0.0,     "max_floor": 15},
}

# Фиксированные этажности сервисов
SERVICE_FIXED_FLOORS = {
    "kindergarten": 2,
    "school": 3,
    "polyclinics": 4,
}

# ---------------- УТИЛИТЫ ----------------

def to_metric_crs(gdf: gpd.GeoDataFrame, like: gpd.GeoDataFrame | None = None, fallback_epsg: int = 3857) -> gpd.GeoDataFrame:
    if like is not None and like.crs is not None:
        return gdf.to_crs(like.crs)
    if gdf.crs is None:
        return gdf.set_crs(epsg=fallback_epsg, allow_override=True)
    if getattr(gdf.crs, "is_projected", None) is True:
        return gdf
    return gdf.to_crs(epsg=fallback_epsg)

def zone_cols(zones: gpd.GeoDataFrame) -> tuple[str, str]:
    zid = "zone_id" if "zone_id" in zones.columns else ("id" if "id" in zones.columns else "ZONE_ID")
    if zid not in zones.columns:
        zones[zid] = np.arange(len(zones))
    zname = "zone"
    if zname not in zones.columns:
        for alt in ["zone_name", "zone_type", "functional_zone_type_name"]:
            if alt in zones.columns:
                zname = alt; break
        else:
            zones["zone"] = "unknown"; zname = "zone"
    return zid, zname

def round_int(x: float) -> int:
    return int(np.floor(x + 0.5))

def waterfill_with_caps(weights: np.ndarray, caps: np.ndarray, demand: float, eps: float = 1e-9) -> np.ndarray:
    n = len(weights)
    x = np.zeros(n, dtype=float)
    remain = np.arange(n)
    D = float(max(demand, 0.0))
    w = weights.copy().astype(float)
    w[w < 0] = 0.0
    while len(remain) > 0 and D > eps:
        sw = float(w[remain].sum())
        if sw <= eps:
            break
        inc = np.zeros_like(x)
        for i in remain:
            quota = D * (w[i] / sw)
            inc[i] = min(quota, caps[i] - x[i])
        inc_sum = float(inc[remain].sum())
        if inc_sum <= eps:
            break
        x += inc
        D -= inc_sum
        remain = np.array([i for i in remain if (caps[i] - x[i]) > eps], dtype=int)
    return x

# ---------------- ЗАГРУЗКА ----------------
buildings = gpd.read_file(BUILDINGS_PATH)
zones = gpd.read_file(ZONES_PATH)
zones = to_metric_crs(zones, like=buildings)

zid_col, zname_col = zone_cols(zones)

# (Пере)назначаем зону, если её нет или есть пропуски
need_zone_join = (zid_col not in buildings.columns) or buildings[zid_col].isna().any()
if need_zone_join:
    cent = buildings.geometry.representative_point()
    j = gpd.sjoin(
        gpd.GeoDataFrame({"i": np.arange(len(buildings))}, geometry=cent, crs=buildings.crs),
        zones[[zid_col, zname_col, "geometry"]],
        how="left", predicate="within"
    ).drop_duplicates("i").set_index("i")[[zid_col, zname_col]]
    buildings = buildings.drop(columns=[zid_col, zname_col], errors="ignore").join(j, how="left")

# нормализуем имя зоны
buildings[zname_col] = buildings[zname_col].astype(str).str.lower().str.strip()
buildings = to_metric_crs(buildings, like=zones)

# Маски по типу
service_series = buildings.get("service").astype(str).str.lower()
is_living  = service_series.eq("living_house")
is_school  = service_series.eq("school")
is_kinderg = service_series.eq("kindergarten")
is_poly    = service_series.eq("polyclinics")

houses = buildings.copy()  # будем писать туда новые колонки для всех объектов

# Геометрическая площадь
houses["area_m2"] = houses.geometry.area.astype(float)

# ---------------- ПАРАМЕТРЫ ЗОН: max_floor и наличие target ----------------
def resolve_zone_params(row) -> tuple[float, int]:
    zid = row.get(zid_col)
    zname = row.get(zname_col)
    targ = None; mf = None
    if pd.notna(zid) and int(zid) in ZONE_PARAMS_BY_ID:
        p = ZONE_PARAMS_BY_ID[int(zid)]
        targ = p.get("target_living_area")
        mf   = p.get("max_floor")
    if (targ is None or mf is None) and pd.notna(zname):
        p = ZONE_PARAMS_BY_NAME.get(str(zname).lower())
        if p:
            if targ is None: targ = p.get("target_living_area")
            if mf   is None: mf   = p.get("max_floor")
    if mf is None: mf = 9  # дефолтная верхняя планка
    return (float(targ) if targ is not None else np.nan), int(mf)

params = houses.apply(resolve_zone_params, axis=1, result_type="expand")
houses["target_zone_area"], houses["max_floor_zone"] = params[0].values, params[1].values

# ---------------- ЭТАЖНОСТЬ ----------------
# 1) ЖИЛЫЕ: floors = round(area/100), clip [1, max_floor_zone]
floors_living = np.maximum(1, np.minimum(
    houses.loc[is_living, "max_floor_zone"].astype(int).values,
    np.vectorize(round_int)(houses.loc[is_living, "area_m2"].values / 100.0)
))
houses.loc[is_living, "floors_count"] = floors_living

# 2) СЕРВИСЫ: фиксированные этажности (и клип по max_floor_zone)
for svc_name, fixed in SERVICE_FIXED_FLOORS.items():
    m = service_series.eq(svc_name)
    if m.any():
        maxf = houses.loc[m, "max_floor_zone"].astype(int)
        houses.loc[m, "floors_count"] = np.minimum(int(fixed), maxf).astype(int)

# Прочим (если есть) ничего не задаём; приведём тип
houses["floors_count"] = pd.to_numeric(houses["floors_count"], errors="coerce").astype("Int64")

# ---------------- РАСПРЕДЕЛЕНИЕ ЖИЛОЙ ПЛОЩАДИ ТОЛЬКО ПО ЗОНАМ С TARGET ----------------
# Для сервисов living_area=0, K=NaN всегда.
houses["K"] = np.nan
houses["living_area"] = 0.0  # по умолчанию 0 (и для зон без target)

# Границы зон для весов
zones["_boundary"] = zones.geometry.boundary

# Группируем только жилые, только по непустому zone_id
valid_living = houses[is_living & houses[zid_col].notna()].copy()

summary_rows = []

for zid, sub in valid_living.groupby(valid_living[zid_col].astype(int)):
    zid_int = int(zid)
    Z = zones.loc[zones[zid_col] == zid_int]
    if len(Z) == 0:
        continue

    # Целевая площадь для этой зоны
    targ = sub["target_zone_area"].dropna().iloc[0] if sub["target_zone_area"].notna().any() else np.nan

    if np.isnan(targ) or targ <= 0:
        # ЯВНО: нет таргета → оставляем living_area=0 и K=NaN
        houses.loc[sub.index, "_alloc_status"] = "no_target → living=0"
        summary_rows.append({
            "zone_id": zid_int,
            "target_requested": float(targ) if not np.isnan(targ) else None,
            "target_used": 0.0,
            "cap_min": float((0.5 * sub["area_m2"] * sub["floors_count"]).sum()),
            "cap_max": float((0.75 * sub["area_m2"] * sub["floors_count"]).sum()),
            "n_buildings": int(len(sub)),
            "mean_K": np.nan,
            "sum_living_area": 0.0,
            "status": "no_target",
        })
        continue

    # Капасити по K
    cap_min = 0.5 * sub["area_m2"] * sub["floors_count"]
    cap_max = 0.75 * sub["area_m2"] * sub["floors_count"]
    cap_min_tot = float(cap_min.sum())
    cap_max_tot = float(cap_max.sum())

    # T в допустимых пределах (чтобы K остался в [0.5,0.75])
    T = float(np.clip(targ, cap_min_tot, cap_max_tot))

    # Веса: ближе к границе — больше
    boundary = unary_union(list(Z["_boundary"].values))
    dists = sub.geometry.representative_point().distance(boundary).astype(float).values
    weights = 1.0 / (1e-6 + dists)
    if not np.isfinite(weights).any() or weights.sum() == 0.0:
        weights = np.ones_like(dists, dtype=float)

    deltas = (cap_max - cap_min).astype(float).values
    D = T - cap_min_tot
    x = waterfill_with_caps(weights, deltas, D)

    living = cap_min.values + x
    denom = (sub["area_m2"].values * sub["floors_count"].values)
    K = np.divide(living, denom, out=np.zeros_like(living), where=denom > 0)
    K = np.clip(K, 0.5, 0.75)

    houses.loc[sub.index, "K"] = K
    houses.loc[sub.index, "living_area"] = living
    status = f"T_used={T:.2f}; cap_min={cap_min_tot:.2f}; cap_max={cap_max_tot:.2f}"
    houses.loc[sub.index, "_alloc_status"] = status

    summary_rows.append({
        "zone_id": zid_int,
        "target_requested": float(targ),
        "target_used": float(T),
        "cap_min": cap_min_tot,
        "cap_max": cap_max_tot,
        "n_buildings": int(len(sub)),
        "mean_K": float(np.mean(K)) if len(K) else np.nan,
        "sum_living_area": float(np.sum(living)),
        "status": status,
    })

# Для сервисов — статус
houses.loc[is_school | is_kinderg | is_poly, "_alloc_status"] = houses.loc[is_school | is_kinderg | is_poly, "_alloc_status"].fillna("service → living=0")

# Собираем сводку
summary_df = pd.DataFrame(summary_rows)

# ---------------- ВЫВОД ----------------
OUT_GEO = BUILDINGS_PATH.with_name(f"{BUILDINGS_PATH.stem}_with_floors_living.geojson")
OUT_CSV = BUILDINGS_PATH.with_name(f"{BUILDINGS_PATH.stem}_living_summary.csv")

# Чистим тех.поля
if "_boundary" in zones.columns:
    zones = zones.drop(columns=["_boundary"])

houses.to_file(OUT_GEO, driver="GeoJSON")
summary_df.to_csv(OUT_CSV, index=False)

print(
    f"OK → {OUT_GEO.name} (floors_count для всех; living_area только в зонах с target)\n"
    f"OK → {OUT_CSV.name} (сводка по зонам с target)\n"
    f"Всего объектов: {len(houses)}, жилых домов: {int(is_living.sum())}, "
    f"средний K по зонам (если есть): {summary_df['mean_K'].mean(skipna=True) if len(summary_df)>0 else float('nan'):.3f}"
)


OK → buildings_clustered_buildings_merged_with_floors_living.geojson (floors_count для всех; living_area только в зонах с target)
OK → buildings_clustered_buildings_merged_living_summary.csv (сводка по зонам с target)
Всего объектов: 253, жилых домов: 246, средний K по зонам (если есть): 0.500
